### Install all required packages:

In [ ]:
!pip install --upgrade pip
! pip install composition_stats
! pip install --upgrade fastsrm
! pip install xgboost
! pip install pandas
! pip install boruta

### Import all required packages and functions:

In [ ]:
#for data manipulation:
import numpy as np
import pandas as pd
import time
import warnings
warnings.filterwarnings("ignore")
from collections import defaultdict
from itertools import chain
#for data preprocessing:
import composition_stats as cs #for clr function
from sklearn.preprocessing import MinMaxScaler
#for model development:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import Lasso
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from boruta import BorutaPy
import xgboost as xgb
#for model performance evaluation and visualization:
from sklearn.metrics import precision_score, recall_score, f1_score, make_scorer, confusion_matrix, roc_auc_score, roc_curve, auc
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification

### Import all required data:

In [ ]:
#import processed genus data from the fransoza dataset:
data_genus = 'fran_genera.csv'
df_genus = pd.read_csv(format(data_genus))
#Rename sample column and index it:
df_genus.rename(columns={'Unnamed: 0': 'Sample'}, inplace=True)
df_genus= df_genus.set_index('Sample')

In [ ]:
#import processed metadata from the fransoza dataset:
data_meta = 'fran_metadata.csv'
df_meta = pd.read_csv(format(data_meta))
#Rename sample column and index it:
df_meta= df_meta.set_index('Sample')
df_meta = df_meta.drop("Unnamed: 0", axis =1)

Set the X variable to the genus feature data and the y variable to the disease group status (IBD or Control):

In [ ]:
X = df_genus
y = df_meta['Disease.Group']

### Preprocessing function:

In [ ]:
#allows the preprocessing within the loop later to be one line compared to many:
def pre_pros(x):
    x_col = list(x.columns)
    x_row = list(x.index)
    as_matrix = x.values
    clr_matrix = cs.clr(as_matrix + 0.00001)
    min_max_matrix = MinMaxScaler().fit_transform(clr_matrix)
    tran_df = pd.DataFrame(min_max_matrix)
    tran_df = tran_df.set_axis(x_row, axis=0)
    tran_df = tran_df.set_axis(x_col, axis=1)
    
    return tran_df

# Model Creation:

#### All the models are in seperate loops due to the limit of processing power and the frequency of crashing when trying to run it all as one loop.

## XGBOOST:

In [ ]:
#set up model:
model_names = ["xgboost"] #allows combination into a larger loop if able
models = {'xgboost': xgb.XGBClassifier(objective = 'binary:logistic', seed= 23)} 

In [ ]:
#set up parameter grid of different parameter values to try in the grid search:
param_grids = {
     'xgboost': {
        'n_estimators': [100, 200, 300, 400, 500],
        'max_depth': [ 10, 20, 30, 40, 50],
        'learning_rate': [0.1, 0.01, 0.05],
        'subsample': [0.8, 0.9, 1.0],
       'colsample_bytree': [0.8, 0.9, 1.0]
   }
}
   

In [ ]:
#create somewhere to store all the performance metrics:
xg_model = {'xgboost' : defaultdict(list)} #only run first time through as will over-ride results otherwise!!

In [ ]:
#run model for all folds of a 10 fold cross validation:
#define the cross fold validation functions:
skf = StratifiedKFold(n_splits=10, shuffle = True, random_state = 23) #for outer split 
skf_small = StratifiedKFold(n_splits=5, shuffle = True, random_state = 23) #for inner split within the grid search
scorer = make_scorer(roc_auc_score, needs_threshold=True) #to use roc score in the grid search
#outer loop which splits the data into 10 cross validation folds
for train_index, test_index in skf.split(X, y):
    #define the training and testing data for the current fold:
    x_train_fold, x_test_fold = X.iloc[train_index], X.iloc[test_index]
    y_train_fold, y_test_fold = y[train_index], y[test_index]

    #pre-process data (scale and normalise):
    x_train_fold = pre_pros(x_train_fold)
    x_test_fold = pre_pros(x_test_fold)
    
    #required numerical y value for xg:
    y_train_fold = y_train_fold.replace({'IBD': 1, 'Control': 0})
    y_test_fold = y_test_fold.replace({'IBD': 1, 'Control': 0})
    
    #perform grid search on the train data:
    for i, MODEL in enumerate(model_names): #allows one loop of models if processing allows
        
        time_start = time.time()
        
        model = models[MODEL] #select the corresponding model
        param_grid = param_grids[MODEL] #select the corresponding parameter grid 
        grid = GridSearchCV(estimator = model, param_grid = param_grid , n_jobs = 1, cv = skf_small, scoring = scorer)
        grid_result = grid.fit(x_train_fold, y_train_fold)
       
        time_grid = time.time() - time_start
        
        print(f"Grid Search took {time_grid/60:.2f} mins ({time_grid:.2f} secs)")
    
        #extract best model from the grid search:
        best_model = grid_result.best_estimator_
        best_model_params = grid_result.best_params_
        best_idx = grid_result.best_index_
        best_grid_model_score = grid_result.cv_results_['mean_test_score'][best_idx]
        best_grid_model_std = grid_result.cv_results_['std_test_score'][best_idx]
        
        #boruta feature selection using the best model:
        feat_selector = BorutaPy(best_model, n_estimators='auto', verbose=2, random_state=1)
        x_train_fold_np  = np.asarray(x_train_fold)
        y_train_fold_np  = np.asarray(y_train_fold)
        feat_selector.fit(x_train_fold_np, y_train_fold_np)
        
        # extract selected features:
        feat_select = feat_selector.support_
        feat_rank = feat_selector.ranking_
        
        #fit new model with selected features and hyperparameters on all training data:
        re_best_model = grid_result.best_estimator_ #new copy of best model
        #select only required features:
        x_train_red = x_train_fold.iloc[:, feat_select]
        x_test_red = x_test_fold.iloc[:, feat_select]
        re_best_model.fit(x_train_red, y_train_fold)
    
        #use best model with selected features on test data:
        y_pred_lab = re_best_model.predict(x_test_red)
        y_pred_prob = re_best_model.predict_proba(x_test_red)
        y_true_lab = y_test_fold
        feature_importance = re_best_model.feature_importances_
        
        #extract performance metrics of best model on test data:
        recall = recall_score(y_true_lab, y_pred_lab)
        precision = precision_score(y_true_lab, y_pred_lab)
        f1 = f1_score(y_true_lab, y_pred_lab)
        confusion = confusion_matrix(y_true_lab, y_pred_lab)
        TP = confusion[1, 1]
        TN = confusion[0, 0]
        FP = confusion[0, 1]
        FN = confusion[1, 0]
        specificity = TN / (TN + FP)
        sensitivity = TP / (TP + FN)

        fpr_num, tpr_num, _ = roc_curve(y_test_fold, y_pred_prob[:,1])
        auc_score = auc(fpr_num, tpr_num)
        
        
        #store metrics:
        xg_model[MODEL]['best_model'].append(best_model)
        xg_model[MODEL]['best_params'].append(best_model_params)
        xg_model[MODEL]['feat_select'].append(feat_select)
        xg_model[MODEL]['feat_rank'].append(feat_rank)
        xg_model[MODEL]['feat_import'].append(feature_importance)
        xg_model[MODEL]['true_lab'].append(y_true_lab)
        xg_model[MODEL]['pred_lab'].append(y_pred_lab)
        xg_model[MODEL]['pred_prob'].append(y_pred_prob)
        xg_model[MODEL]['recall'].append(recall)
        xg_model[MODEL]['precision'].append(precision)
        xg_model[MODEL]['f1'].append(f1)
        xg_model[MODEL]['specificity'].append(specificity)
        xg_model[MODEL]['sensitivity'].append(sensitivity)
        xg_model[MODEL]['best_mod_auc'].append(best_grid_model_score)
        xg_model[MODEL]['best_mod_std'].append(best_grid_model_std)
        xg_model[MODEL]['test_auc'].append(auc_score)
        
        print(xg_model[MODEL]) #allows you to idenitfy when a loop is complete and view that loops metrics 
        
        %store xg_model #allows you to extract data later without having to re-run the whole loop

## Random Forest:

In [ ]:
#set up model:
model_names = ["rf"] #allows combination into a larger loop if able
models = {'rf': RandomForestClassifier(random_state=23)}

In [ ]:
#set up parameter grid of different parameter values to try in the grid search:
param_grids = {
       'rf': {
       'n_estimators' : [100, 200, 300, 400, 500],
        'criterion' : ['gini', 'entropy'],
        'max_depth' : [10, 20, 30, 40, 50],
        'max_features' : [ 0.25, 0.5, 0.75, 1.0], 
        'max_samples': [0.8, 0.9, 1.0]
        
    }
}

In [ ]:
#create somewhere to store all the performance metrics:
rf_model = {'rf':defaultdict(list)} #only run first time through as will over-ride results otherwise!!

In [ ]:
#run model for all folds of a 10 fold cross validation:
#define the cross fold validation functions:
skf = StratifiedKFold(n_splits=10, shuffle = True, random_state = 23) #for outer split 
skf_small = StratifiedKFold(n_splits=5, shuffle = True, random_state = 23) #for inner split within the grid search
scorer = make_scorer(roc_auc_score, needs_threshold=True) #to use roc score in the grid search
#outer loop which splits the data into 10 cross validation folds:
for train_index, test_index in skf.split(X, y):
    #define the training and testing data for the current fold:
    x_train_fold, x_test_fold = X.iloc[train_index], X.iloc[test_index]
    y_train_fold, y_test_fold = y[train_index], y[test_index]

    #pre-process data (scale and normalise):
    x_train_fold = pre_pros(x_train_fold)
    x_test_fold = pre_pros(x_test_fold)
    
    #make y vector numerical:
    y_train_fold = y_train_fold.replace({'IBD': 1, 'Control': 0})
    y_test_fold = y_test_fold.replace({'IBD': 1, 'Control': 0})
    
    #perform grid search on the train data:
    for i, MODEL in enumerate(model_names): #allows one loop for all models if processing allows
        
        time_start = time.time()
        
        model = models[MODEL] #select the corresponding model
        param_grid = param_grids[MODEL] #select the corresponding parameter grid
        grid = GridSearchCV(estimator = model, param_grid = param_grid , n_jobs = 1, cv = skf_small, scoring = scorer)
        grid_result = grid.fit(x_train_fold, y_train_fold)
       
        time_grid = time.time() - time_start
        
        print(f"Grid Search took {time_grid/60:.2f} mins ({time_grid:.2f} secs)")
    
        #extract best model from the grid search:
        best_model = grid_result.best_estimator_
        best_model_params = grid_result.best_params_
        best_idx = grid_result.best_index_
        best_grid_model_score = grid_result.cv_results_['mean_test_score'][best_idx]
        best_grid_model_std = grid_result.cv_results_['std_test_score'][best_idx]
        
        #Boruta feature selection on best model:
        feat_selector = BorutaPy(best_model, n_estimators='auto', verbose=2, random_state=1)
        x_train_fold_np  = np.asarray(x_train_fold)
        y_train_fold_np  = np.asarray(y_train_fold)
        feat_selector.fit(x_train_fold_np, y_train_fold_np)
        
        # extract selected features:
        feat_select = feat_selector.support_
        feat_rank = feat_selector.ranking_
        
        #fit new model with selected features and hyperparameters on all training data:
        re_best_model = grid_result.best_estimator_ #new copy of best model
        #extract only selected features:
        x_train_red = x_train_fold.iloc[:, feat_select]
        x_test_red = x_test_fold.iloc[:, feat_select]
        re_best_model.fit(x_train_red, y_train_fold)
    
        #use best model with selected features on test data:
        y_pred_lab = re_best_model.predict(x_test_red)
        y_pred_prob = re_best_model.predict_proba(x_test_red)
        y_true_lab = y_test_fold
        feature_importance = re_best_model.feature_importances_
        
        #extract performance metrics of best model on test data:
        recall = recall_score(y_true_lab, y_pred_lab)
        precision = precision_score(y_true_lab, y_pred_lab)
        f1 = f1_score(y_true_lab, y_pred_lab)
        confusion = confusion_matrix(y_true_lab, y_pred_lab)
        TP = confusion[1, 1]
        TN = confusion[0, 0]
        FP = confusion[0, 1]
        FN = confusion[1, 0]
        specificity = TN / (TN + FP)
        sensitivity = TP / (TP + FN)

        fpr_num, tpr_num, _ = roc_curve(y_test_fold, y_pred_prob[:,1])
        auc_score = auc(fpr_num, tpr_num)
        
        
        #store metrics:
        rf_model[MODEL]['best_model'].append(best_model)
        rf_model[MODEL]['best_params'].append(best_model_params)
        rf_model[MODEL]['feat_select'].append(feat_select)
        rf_model[MODEL]['feat_rank'].append(feat_rank)
        rf_model[MODEL]['feat_import'].append(feature_importance)
        rf_model[MODEL]['true_lab'].append(y_true_lab)
        rf_model[MODEL]['pred_lab'].append(y_pred_lab)
        rf_model[MODEL]['pred_prob'].append(y_pred_prob)
        rf_model[MODEL]['recall'].append(recall)
        rf_model[MODEL]['precision'].append(precision)
        rf_model[MODEL]['f1'].append(f1)
        rf_model[MODEL]['specificity'].append(specificity)
        rf_model[MODEL]['sensitivity'].append(sensitivity)
        rf_model[MODEL]['best_mod_auc'].append(best_grid_model_score)
        rf_model[MODEL]['best_mod_std'].append(best_grid_model_std)
        rf_model[MODEL]['test_auc'].append(auc_score)
        
        print(rf_model[MODEL]) #allows you to idenitfy when a loop is complete and view that loops metrics 
        
        %store rf_model #allows you to extract data later without having to re-run the whole loop

## LASSO:

In [ ]:
#set up model:
model_names = ["lasso"]  #allows combination into a larger loop if able
models = {'lasso': Lasso(random_state = 23) }

In [ ]:
#set up parameter grid of different parameter values to try in the grid search:
param_grids = {
       'lasso': {
       'alpha' : list(np.logspace(-4,2,7))   
    }
}

In [ ]:
#create somewhere to store all the performance metrics:
lasso_model = {'lasso':defaultdict(list)} #only run first time through as will over-ride results otherwise!!

In [ ]:
#run model for all folds of a 10 fold cross validation:
#define the cross fold validation functions:
skf = StratifiedKFold(n_splits=10, shuffle = True, random_state = 23) #for outer split 
skf_small = StratifiedKFold(n_splits=5, shuffle = True, random_state = 23) #for inner split within the grid search
scorer = make_scorer(roc_auc_score, needs_threshold=True) #to use roc score in the grid search
#outer loop which splits the data into 10 cross validation folds:
for train_index, test_index in skf.split(X, y):
    #define the training and testing data for the current fold:
    x_train_fold, x_test_fold = X.iloc[train_index], X.iloc[test_index]
    y_train_fold, y_test_fold = y[train_index], y[test_index]

    #pre-process data (scale and normalise):
    x_train_fold = pre_pros(x_train_fold)
    x_test_fold = pre_pros(x_test_fold)
    
    #make y vector numerical:
    y_train_fold = y_train_fold.replace({'IBD': 1, 'Control': 0})
    y_test_fold = y_test_fold.replace({'IBD': 1, 'Control': 0})
    
    #perform grid search on the train data:
    for i, MODEL in enumerate(model_names): #allows one loop for all models if processing allows
        
        time_start = time.time()
        
        model = models[MODEL] #select corresponding model
        param_grid = param_grids[MODEL] #select corresponding parameter grid
        grid = GridSearchCV(estimator = model, param_grid = param_grid , n_jobs = 1, cv = skf_small, scoring = scorer)
        grid_result = grid.fit(x_train_fold, y_train_fold)
       
        time_grid = time.time() - time_start
        
        print(f"Grid Search took {time_grid/60:.2f} mins ({time_grid:.2f} secs)")
    
        #extract best model from the grid search:
        best_model = grid_result.best_estimator_
        best_model_params = grid_result.best_params_
        best_idx = grid_result.best_index_
        best_grid_model_score = grid_result.cv_results_['mean_test_score'][best_idx]
        best_grid_model_std = grid_result.cv_results_['std_test_score'][best_idx]
    
        #extract important features as selected by the built in LASSO model:
        feat_select = (best_model.coef_ != 0)
    
        #use best model parameters and features on test data:
        y_pred_prob = best_model.predict(x_test_fold)
        y_pred_lab = np.where(y_pred_prob >= 0.5, 1, 0) #defined as IBD if probability if over 0.5 
        y_true_lab = y_test_fold
        
        #extract performance metrics of best model on test data:
        recall = recall_score(y_true_lab, y_pred_lab)
        precision = precision_score(y_true_lab, y_pred_lab)
        f1 = f1_score(y_true_lab, y_pred_lab)
        confusion = confusion_matrix(y_true_lab, y_pred_lab)
        TP = confusion[1, 1]
        TN = confusion[0, 0]
        FP = confusion[0, 1]
        FN = confusion[1, 0]
        specificity = TN / (TN + FP)
        sensitivity = TP / (TP + FN)

        fpr_num, tpr_num, _ = roc_curve(y_test_fold, y_pred_prob)
        auc_score = auc(fpr_num, tpr_num)
        
        
        #store metrics:
        lasso_model[MODEL]['best_model'].append(best_model)
        lasso_model[MODEL]['best_params'].append(best_model_params)
        lasso_model[MODEL]['true_lab'].append(y_true_lab)
        lasso_model[MODEL]['pred_lab'].append(y_pred_lab)
        lasso_model[MODEL]['pred_prob'].append(y_pred_prob)
        lasso_model[MODEL]['recall'].append(recall)
        lasso_model[MODEL]['precision'].append(precision)
        lasso_model[MODEL]['feat_select'].append(feat_select)
        lasso_model[MODEL]['f1'].append(f1)
        lasso_model[MODEL]['specificity'].append(specificity)
        lasso_model[MODEL]['sensitivity'].append(sensitivity)
        lasso_model[MODEL]['best_mod_auc'].append(best_grid_model_score)
        lasso_model[MODEL]['best_mod_std'].append(best_grid_model_std)
        lasso_model[MODEL]['test_auc'].append(auc_score)
        
        print(lasso_model[MODEL]) #allows you to idenitfy when a loop is complete and view that loops metrics 
        
        %store lasso_model  #allows you to extract data later without having to re-run the whole loop

# Extract feature names and visualize model performance:

### Extract model performance metrics from store:

In [1]:
store -r lasso_model

In [2]:
store -r rf_model

In [3]:
store -r xg_model

### Extract feature names:

In [ ]:
lasso_features = []
#append features names to a list from the data for all folds:
for fold in range(0,10):
    imp_feat = X.iloc[:,(lasso_model['lasso']['feat_select'][fold] == True)]
    features = imp_feat.columns
    
    lasso_features.append(list(features))
    print(features)

In [ ]:
#combine all features from all folds into one big list:
lasso_features = list(chain(*lasso_features))
#extract the names of all the unique features:
unique_lasso = np.unique(lasso_features)
#find the number of unique features:
len(unique_lasso)

In [ ]:
rf_features = []
#append features names to a list from the data for all folds:
for fold in range(0,10):
    imp_feat = X.iloc[:,(rf_model['rf']['feat_select'][fold] == True)]
    features = imp_feat.columns
    
    rf_features.append(list(features))
    print(features)

In [ ]:
#combine all features from all folds into one big list:
rf_features = list(chain(*rf_features))
#extract the names of all the unique features:
unique_rf = np.unique(rf_features)
#find the number of unique features:
len(unique_rf)

In [ ]:
xg_features = []
#append feature names to a list from the data for all folds:
for fold in range(0,10):
    imp_feat = X.iloc[:,(xg_model['xgboost']['feat_select'][fold] == True)]
    features = imp_feat.columns
    
    xg_features.append(list(features))
    print(features)

In [ ]:
#combine all features from all folds into one big list:
xg_features = list(chain(*xg_features))
#extract the names of all the unique features:
unique_xg = np.unique(xg_features)
#find the number of unique features:
len(unique_xg)

In [ ]:
#obtain the number of features that are identified by more than one model:
len(set(unique_xg) & set(unique_rf) & set(unique_lasso))
len((set(unique_rf) & set(unique_xg)))
len((set(unique_xg) & set(unique_lasso)))
len((set(unique_rf) & set(unique_lasso)))

### Extract the average and sd of the performance metrics across the 10 folds:

In [ ]:
#xgboost averages:
print(np.mean(xg_model['xgboost']['recall']))
print(np.mean(xg_model['xgboost']['precision']))
print(np.mean(xg_model['xgboost']['f1']))
print(np.mean(xg_model['xgboost']['specificity']))
print(np.mean(xg_model['xgboost']['sensitivity']))
print(np.mean(xg_model['xgboost']['test_auc']))
#xgboost sds:
print(np.std(xg_model['xgboost']['recall']))
print(np.std(xg_model['xgboost']['precision']))
print(np.std(xg_model['xgboost']['f1']))
print(np.std(xg_model['xgboost']['specificity']))
print(np.std(xg_model['xgboost']['sensitivity']))
print(np.std(xg_model['xgboost']['test_auc']))

In [ ]:
#random forest averages:
print(np.mean(rf_model['rf']['recall']))
print(np.mean(rf_model['rf']['precision']))
print(np.mean(rf_model['rf']['f1']))
print(np.mean(rf_model['rf']['specificity']))
print(np.mean(rf_model['rf']['sensitivity']))
print(np.mean(rf_model['rf']['test_auc']))
#random forest sds:
print(np.std(rf_model['rf']['recall']))
print(np.std(rf_model['rf']['precision']))
print(np.std(rf_model['rf']['f1']))
print(np.std(rf_model['rf']['specificity']))
print(np.std(rf_model['rf']['sensitivity']))
print(np.std(rf_model['rf']['test_auc']))

In [ ]:
#lasso averages:
print(np.mean(lasso_model['lasso']['recall']))
print(np.mean(lasso_model['lasso']['precision']))
print(np.mean(lasso_model['lasso']['f1']))
print(np.mean(lasso_model['lasso']['specificity']))
print(np.mean(lasso_model['lasso']['sensitivity']))
print(np.mean(lasso_model['lasso']['test_auc']))
#lasso sds:
print(np.std(lasso_model['lasso']['recall']))
print(np.std(lasso_model['lasso']['precision']))
print(np.std(lasso_model['lasso']['f1']))
print(np.std(lasso_model['lasso']['specificity']))
print(np.std(lasso_model['lasso']['sensitivity']))
print(np.std(lasso_model['lasso']['test_auc']))

### ROC curve of all the models:

In [ ]:
#create lists to store true positive rates and set false positive rates:
tprs_rf = []
base_fpr_rf = np.linspace(0, 1, 101)
tprs_xg = []
base_fpr_xg = np.linspace(0, 1, 101)
tprs_lasso = []
base_fpr_lasso = np.linspace(0, 1, 101)

#set up blank figure space:
plt.figure(figsize=(8, 8))
plt.axes().set_aspect('equal', 'datalim')

#calculate the corresponding tprs for the fprs of each fold of each model:
for i in range(0,10):
    fpr, tpr, _ = roc_curve(rf_model['rf']['true_lab'][i], rf_model['rf']['pred_prob'][i][:,1])
    
    tpr = np.interp(base_fpr_rf, fpr, tpr)
    tpr[0] = 0.0
    tprs_rf.append(tpr)

for i in range(0,10):
    fpr, tpr, _ = roc_curve(lasso_model['lasso']['true_lab'][i], lasso_model['lasso']['pred_prob'][i])
    
    tpr = np.interp(base_fpr_lasso, fpr, tpr)
    tpr[0] = 0.0
    tprs_lasso.append(tpr)

for i in range(0,10):
    fpr, tpr, _ = roc_curve(xg_model['xgboost']['true_lab'][i], xg_model['xgboost']['pred_prob'][i][:,1])
    
    tpr = np.interp(base_fpr_xg, fpr, tpr)
    tpr[0] = 0.0
    tprs_xg.append(tpr)
    
#find the average and sd of the tprs for each fold of each model:
tprs_rf = np.array(tprs_rf)
mean_tprs_rf = tprs_rf.mean(axis=0)
std_rf = tprs_rf.std(axis=0)

tprs_lasso = np.array(tprs_lasso)
mean_tprs_lasso = tprs_lasso.mean(axis=0)
std_lasso = tprs_lasso.std(axis=0)

tprs_xg = np.array(tprs_xg)
mean_tprs_xg = tprs_xg.mean(axis=0)
std_xg = tprs_xg.std(axis=0)

#find the confidence interval for the average tprs of each model:
tprs_upper_rf = np.minimum(mean_tprs_rf + std_rf, 1)
tprs_lower_rf = mean_tprs_rf - std_rf
tprs_upper_xg = np.minimum(mean_tprs_xg + std_xg, 1)
tprs_lower_xg = mean_tprs_xg - std_xg
tprs_upper_lasso = np.minimum(mean_tprs_lasso + std_lasso, 1)
tprs_lower_lasso = mean_tprs_lasso - std_lasso

#calculate the area under the roc for each of the average models performance:
auc_rf = auc(base_fpr_rf, mean_tprs_rf)
auc_xg = auc(base_fpr_xg, mean_tprs_xg)
auc_lasso = auc(base_fpr_lasso, mean_tprs_lasso)

#plot the average roc for each model with the confidence interval surrounding:
plt.plot(base_fpr_rf, mean_tprs_rf, 'palevioletred', label=r'Mean ROC RandomForest (AUC = {} $\pm$ {})'.format(round(auc_rf,2), 0.04))
plt.fill_between(base_fpr_rf, tprs_lower_rf, tprs_upper_rf, color='pink', alpha=0.3)
plt.plot(base_fpr_xg, mean_tprs_xg, 'darkcyan', label=r'Mean ROC XGBoost (AUC = {} $\pm$ {})'.format(round(auc_xg,2), 0.05))
plt.fill_between(base_fpr_xg, tprs_lower_xg, tprs_upper_xg, color='deepskyblue', alpha=0.3)
plt.plot(base_fpr_lasso, mean_tprs_lasso, 'orange', label=r'Mean ROC LASSO (AUC = {} $\pm$ {})'.format(round(auc_lasso,2), 0.04))
plt.fill_between(base_fpr_lasso, tprs_lower_lasso, tprs_upper_lasso, color='sandybrown', alpha=0.2)

#plot on the blank figure space and add title labels:
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([-0.01, 1.01])
plt.ylim([-0.01, 1.01])
plt.legend(loc="lower right")
plt.title('Receiver Operating Characteristic Microbiome Model Averages')
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.savefig('micro_avg.pdf', format = 'pdf', dpi = 300, bbox_inches = 'tight')
plt.show()